In [1]:
# Importing Basic libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import re
# Importing time series specific libraries
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller
from scipy.stats import bartlett
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima.model import ARIMA
import pmdarima
from pmdarima import auto_arima
from statsmodels.tsa.statespace import sarimax
import prophet
from prophet import Prophet
from scipy.stats import levene
# Miscellaneous libararies
import warnings
warnings.filterwarnings('ignore')
from math import sqrt
from random import random

# Libaraies for evaluation of model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, mean_squared_log_error
from statsmodels.tsa.arima.model import ARIMAResults


c:\Users\milla\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#TODO Cargar los datos de la hoja de calculo en el archivo "load_IJV.xlsx"
# Obtener la ruta absoluta del script de Python
def importar_datos(x):
    global df
    ruta_script = os.getcwd()
    #Nombre del archivo de Excel a cargar
    nombre_archivo = x
    # Combinar la ruta del script y el nombre del archivo
    ruta_archivo = os.path.join(ruta_script, nombre_archivo)
    # Cargar los datos del archivo de Excel en un DataFrame de pandas
    df = pd.read_excel(ruta_archivo)
    # Imprimir el df 
    return df
importar_datos('load_IJV.xlsx')
print(df)

     idprov      fecha  dem1  dem2  dem3  dem4  dem5  dem6  dem7  dem8  ...  \
0       IJV 2007-05-01   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
1       IJV 2007-07-01   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
2       IJV 2007-07-27   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
3       IJV 2007-07-28   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4       IJV 2007-07-30   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
...     ...        ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
3915    IJV 2018-04-14  15.0  14.0  13.0  13.0  13.0  13.0  13.0  12.0  ...   
3916    IJV 2018-04-15  15.0  14.0  14.0  13.0  13.0  13.0  12.0  12.0  ...   
3917    IJV 2018-04-16  14.0  13.0  13.0  13.0  12.0  13.0  12.0  12.0  ...   
3918    IJV 2018-04-17  11.0  11.0  10.0  10.0  10.0  10.0  10.0  10.0  ...   
3919    IJV 2018-04-18  12.0  11.0  11.0  10.0  10.0  11.0  11.0  11.0  ...   

      dem16  dem17  dem18  dem19  dem20  dem21  dem

In [3]:
# #Identificar los valores únicos en la columna "idprov"
# idprov_unique = df["idprov"].unique()

# # Verificar si hay valores diferentes a "IJV"
# if len(idprov_unique) > 1 or idprov_unique[0] != "'IJV'":
#     # Mostrar los valores diferentes a "IJV"
#     print("Los siguientes valores no son 'IJV':")
#     print(df.loc[df["idprov"] != "IJV"])
# else:
#     # Todos los valores de la columna "idprov" son "IJV"
#     print("Todos los valores de la columna 'idprov' son 'IJV'")
#     # Eliminar la columna "idprov" del DataFrame
#     df = df.drop("idprov", axis=1)
# print(df)
df = df[df['fecha'] >= '2008-01-01']
df = df[df['fecha'] <= '2017-12-31']
print(df)

     idprov      fecha  dem1  dem2  dem3  dem4  dem5  dem6  dem7  dem8  ...  \
159     IJV 2008-01-01   9.0   9.0   8.0   8.0   8.0   9.0   9.0  10.0  ...   
160     IJV 2008-01-02   8.0   6.0   8.0   7.0   8.0   8.0   8.0  10.0  ...   
161     IJV 2008-01-03   6.0   4.0   5.0   6.0   6.0   5.0  10.0  10.0  ...   
162     IJV 2008-01-04   6.0   6.0   7.0   6.0   6.0   8.0  10.0  11.0  ...   
163     IJV 2008-01-05   6.0   6.0   6.0   6.0   6.0   8.0   9.0  10.0  ...   
...     ...        ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
3807    IJV 2017-12-27  10.0  10.0   9.0   9.0  10.0  10.0  11.0  12.0  ...   
3808    IJV 2017-12-28  10.0  10.0  10.0   9.0  10.0  10.0  11.0  12.0  ...   
3809    IJV 2017-12-29  11.0  10.0  10.0  10.0  10.0  10.0  11.0  11.0  ...   
3810    IJV 2017-12-30  11.0  11.0  10.0  10.0  10.0  10.0  11.0  11.0  ...   
3811    IJV 2017-12-31  11.0  10.0  10.0  10.0   9.0  10.0  10.0  11.0  ...   

      dem16  dem17  dem18  dem19  dem20  dem21  dem

In [4]:
# eliminar la columna 25 porque me da el maximo valor solamente 
df = df.drop('dem25', axis=1)
print(df)

     idprov      fecha  dem1  dem2  dem3  dem4  dem5  dem6  dem7  dem8  ...  \
159     IJV 2008-01-01   9.0   9.0   8.0   8.0   8.0   9.0   9.0  10.0  ...   
160     IJV 2008-01-02   8.0   6.0   8.0   7.0   8.0   8.0   8.0  10.0  ...   
161     IJV 2008-01-03   6.0   4.0   5.0   6.0   6.0   5.0  10.0  10.0  ...   
162     IJV 2008-01-04   6.0   6.0   7.0   6.0   6.0   8.0  10.0  11.0  ...   
163     IJV 2008-01-05   6.0   6.0   6.0   6.0   6.0   8.0   9.0  10.0  ...   
...     ...        ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
3807    IJV 2017-12-27  10.0  10.0   9.0   9.0  10.0  10.0  11.0  12.0  ...   
3808    IJV 2017-12-28  10.0  10.0  10.0   9.0  10.0  10.0  11.0  12.0  ...   
3809    IJV 2017-12-29  11.0  10.0  10.0  10.0  10.0  10.0  11.0  11.0  ...   
3810    IJV 2017-12-30  11.0  11.0  10.0  10.0  10.0  10.0  11.0  11.0  ...   
3811    IJV 2017-12-31  11.0  10.0  10.0  10.0   9.0  10.0  10.0  11.0  ...   

      dem15  dem16  dem17  dem18  dem19  dem20  dem

In [5]:
#  #convertir la columna 'fecha' a formato de fecha y hora
# df['fecha'] = pd.to_datetime(df['fecha'], format="'%d-%b-%Y %H:%M:%S'")
# # convertir la columna 'fecha' a formato de fecha y hora
# df['fecha'] = pd.to_datetime(df['fecha'])
# # establecer la columna 'fecha' como el índice del DataFrame
# df = df.set_index('fecha')
# # utilizar resample con una frecuencia de 1 hora para agregar una fila por cada hora del día
# df_resampled = df.resample('1H').asfreq()
# display(df_resampled)
# restaring the index
df.index = np.arange(1, len(df) + 1, 1)
print(df.head())

  idprov      fecha  dem1  dem2  dem3  dem4  dem5  dem6  dem7  dem8  ...  \
1    IJV 2008-01-01   9.0   9.0   8.0   8.0   8.0   9.0   9.0  10.0  ...   
2    IJV 2008-01-02   8.0   6.0   8.0   7.0   8.0   8.0   8.0  10.0  ...   
3    IJV 2008-01-03   6.0   4.0   5.0   6.0   6.0   5.0  10.0  10.0  ...   
4    IJV 2008-01-04   6.0   6.0   7.0   6.0   6.0   8.0  10.0  11.0  ...   
5    IJV 2008-01-05   6.0   6.0   6.0   6.0   6.0   8.0   9.0  10.0  ...   

   dem15  dem16  dem17  dem18  dem19  dem20  dem21  dem22  dem23  dem24  
1   11.0   11.0   12.0   14.0   15.0   12.0   11.0   11.0   10.0   10.0  
2    8.0   12.0   15.0   17.0   16.0   11.0    9.0    8.0    7.0    6.0  
3   11.0   11.0   15.0   18.0   17.0   12.0   10.0    9.0    7.0    7.0  
4   11.0   12.0   14.0   18.0   19.0   13.0   10.0    9.0    7.0    7.0  
5   11.0   12.0   11.0   18.0   19.0   14.0   11.0   10.0    8.0    8.0  

[5 rows x 26 columns]


In [6]:
# def sort_columns(col):
#     return int(re.findall(r'\d+', col)[0])

# # ordenar las columnas de demanda por su número
# cols_dem = sorted([col for col in df_resampled.columns if 'dem' in col], key=sort_columns)


# # apilar las columnas de demanda en una sola columna
# df_melted = df_resampled[cols_dem].reset_index().melt(id_vars='fecha', var_name='hora', value_name='demanda')

# # eliminar las filas con NaN en la columna de demanda
# df_clean = df_melted.dropna(subset=['demanda'])

# # convertir la columna hora en una categoría ordenada
# df_clean['hora'] = pd.Categorical(df_clean['hora'], categories=[f'dem{i}' for i in range(0,25)], ordered=True)


# # ordenar el DataFrame por fecha y hora
# df_final = df_clean.sort_values(['fecha', 'hora'])



# # resetear el índice del DataFrame
# df_final = df_final.reset_index(drop=True)

# df_final = df_final.drop('hora', axis=1)

# #TODO ponerle hora a cada una de las fechas -------------------------------------------------------------------------------------
# # convertir la columna fecha en un objeto datetime
# df_final['fecha'] = pd.to_datetime(df_final['fecha'], format='%d-%b-%Y %H:%M:%S')

# last_day = None
# # Organizar el datatime para agregarle una hora a cada datatime cada 24 horas
# for i in range(len(df_final)):
#     # Verificar si se debe reiniciar la suma
#     if last_day is None or df_final.loc[i, 'fecha'].day != last_day:
#         last_day = df_final.loc[i, 'fecha'].day
#         hours_to_add = 1
#     else:
#         hours_to_add += 1
        
#     # Agregar la cantidad de horas correspondiente
#     df_final.loc[i, 'fecha'] += pd.DateOffset(hours=hours_to_add)

# display( df_final) 
# removing February 29
print("lengthBefore =", len(df)) # checking
idx_1 = df[df["fecha"] == '2008-02-29']
idx_2 = df[df["fecha"] == '2012-02-29']
idx_3 = df[df["fecha"] == '2016-02-29']
toRemove = [idx_1.index.item(), idx_2.index.item(), idx_3.index.item()]
print(toRemove)
df = df.drop(toRemove)
print("lengthAfter =", len(df)) # checking

lengthBefore = 3653
[60, 1521, 2982]
lengthAfter = 3650


In [7]:
# # Seleccionar las filas que no tienen fechas del 29 de febrero
# df_final = df_final.loc[(df_final['fecha'].dt.day != 29) | (df_final['fecha'].dt.month != 2)]

# # Imprimir el nuevo DataFrame sin las filas del 29 de febrero
# print(df_final)
# filtering data needed
dfFiltered = df[df['fecha'].dt.strftime('%m-%d') == '01-01']
print(dfFiltered)

     idprov      fecha  dem1  dem2  dem3  dem4  dem5  dem6  dem7  dem8  ...  \
1       IJV 2008-01-01   9.0   9.0   8.0   8.0   8.0   9.0   9.0  10.0  ...   
367     IJV 2009-01-01   8.0   8.0   8.0   7.0   7.0   7.0   7.0   8.0  ...   
732     IJV 2010-01-01   8.0   7.0   7.0   7.0   7.0   7.0   7.0   8.0  ...   
1097    IJV 2011-01-01   7.0   6.0   6.0   6.0   6.0   6.0   6.0   8.0  ...   
1462    IJV 2012-01-01   8.0   7.0   6.0   7.0   7.0   7.0   7.0   8.0  ...   
1828    IJV 2013-01-01   8.0   8.0   8.0   7.0   7.0   7.0   7.0   8.0  ...   
2193    IJV 2014-01-01  11.0  10.0  10.0  10.0   9.0  10.0  10.0  10.0  ...   
2558    IJV 2015-01-01  10.0  10.0   9.0   9.0   9.0   9.0   9.0  10.0  ...   
2923    IJV 2016-01-01  12.0  12.0  11.0  11.0  11.0  11.0  10.0  11.0  ...   
3289    IJV 2017-01-01  10.0  10.0  10.0   9.0   9.0   9.0   9.0  10.0  ...   

      dem15  dem16  dem17  dem18  dem19  dem20  dem21  dem22  dem23  dem24  
1      11.0   11.0   12.0   14.0   15.0   12.0   11.0

In [9]:
# # TODO calcular los valores atipicos de los datos a partir del cálculo basado en el rango intercuartílico (RIQ)
df_final = dfFiltered
# restaring the index
df_final.index = np.arange(1, len(df_final) + 1, 1)
print(df_final)

# reshape the data
dataToFit = []
for k in range(1, 11):
    print(k)
    data = df_final.loc[k,['dem1', 'dem2', 'dem3', 'dem4', 'dem5', 'dem6', 'dem7', 'dem8', 'dem9', 'dem10', 'dem11', 'dem12', 'dem13', 'dem14', 'dem15', 'dem16', 'dem17', 'dem18', 'dem19', 'dem20', 'dem21', 'dem22', 'dem23', 'dem24']]
    data = np.array(data)    
    dataToFit.append(data)
    
    #Calcula el rango intercuartílico (RIQ):
Q1 = np.percentile(dataToFit, 25)
Q3 = np.percentile(dataToFit, 75)
RIQ = Q3 - Q1

#Define los límites inferior y superior para detectar outliers:
limite_inferior = Q1 - 1.5 * RIQ
limite_superior = Q3 + 1.5 * RIQ

#Identifica los outliers:
outliers1 = dataToFit < limite_inferior
outliers2 = dataToFit > limite_superior
dataToFit = np.array(dataToFit)
dataToFit = dataToFit.flatten()
dataToFit = np.float64(dataToFit)
print(dataToFit)
print('outliers1  = ', outliers1)
print('outliers2  = ', outliers2)

   idprov      fecha  dem1  dem2  dem3  dem4  dem5  dem6  dem7  dem8  ...  \
1     IJV 2008-01-01   9.0   9.0   8.0   8.0   8.0   9.0   9.0  10.0  ...   
2     IJV 2009-01-01   8.0   8.0   8.0   7.0   7.0   7.0   7.0   8.0  ...   
3     IJV 2010-01-01   8.0   7.0   7.0   7.0   7.0   7.0   7.0   8.0  ...   
4     IJV 2011-01-01   7.0   6.0   6.0   6.0   6.0   6.0   6.0   8.0  ...   
5     IJV 2012-01-01   8.0   7.0   6.0   7.0   7.0   7.0   7.0   8.0  ...   
6     IJV 2013-01-01   8.0   8.0   8.0   7.0   7.0   7.0   7.0   8.0  ...   
7     IJV 2014-01-01  11.0  10.0  10.0  10.0   9.0  10.0  10.0  10.0  ...   
8     IJV 2015-01-01  10.0  10.0   9.0   9.0   9.0   9.0   9.0  10.0  ...   
9     IJV 2016-01-01  12.0  12.0  11.0  11.0  11.0  11.0  10.0  11.0  ...   
10    IJV 2017-01-01  10.0  10.0  10.0   9.0   9.0   9.0   9.0  10.0  ...   

    dem15  dem16  dem17  dem18  dem19  dem20  dem21  dem22  dem23  dem24  
1    11.0   11.0   12.0   14.0   15.0   12.0   11.0   11.0   10.0   10.0  
2 

In [ ]:
# Reemplazar los valores atípicos con NaN en el DataFrame original
df_final.loc[df_final['fecha'].isin(outliers['fecha']), 'demanda'] = np.nan

# Imprimir el DataFrame con los valores atípicos reemplazados con NaN
display(df_final)

In [ ]:
# Agrupar los valores por día del año y calcular el promedio de los valores por día del año utilizando la función transform
promedios_dia = df_final.groupby([df_final['fecha'].dt.month, df_final['fecha'].dt.day])['demanda'].transform(lambda x: x.mean())
promedios_dia=promedios_dia.round(decimals=0).astype(int)
# Rellenar los valores NaN en la columna de demanda con los promedios calculados utilizando la función fillna
df_final['demanda'].fillna(promedios_dia, inplace=True)

# Imprimir el DataFrame actualizado
print(df_final)

In [ ]:
 #convertir la columna 'fecha' a formato de fecha y hora
df_final['fecha'] = pd.to_datetime(df_final['fecha'], format="'%d-%b-%Y %H:%M:%S'")
# convertir la columna 'fecha' a formato de fecha y hora
df_final['fecha'] = pd.to_datetime(df_final['fecha'])
# establecer la columna 'fecha' como el índice del DataFrame
df_final = df_final.set_index('fecha')
# Crear una figura y un eje
fig, ax = plt.subplots()

# Graficar los datos
demanda=df_final['demanda'].values
ax.plot(df_final.index,demanda )

# Establecer la frecuencia y el formato de los valores del eje x
ax.set_xticks(df_final.index[::8928])
ax.set_xticklabels(df_final.index[::8928], rotation=45)

# Agregar etiquetas a los ejes
ax.set_xlabel('Fecha')
ax.set_ylabel('Demanda')

# Agregar un título al gráfico
ax.set_title('Título del gráfico')

# Mostrar el gráfico
plt.show()
df_final

In [ ]:
# Acotar el DataFrame a un rango de fechas específico
fecha_inicial = '2007-01-01 00:00:00'
fecha_final = '2018-01-01 00:00:00'
df_acotado = df_final.loc[(df_final.index >= fecha_inicial) & (df_final.index <= fecha_final)]
# Resetear el índice y convertir la columna de fechas en una columna del DataFrame
df_acotado = df_acotado.reset_index(drop=False)

# Renombrar la columna 'index' a un nombre más apropiado
df = df.rename(columns={'index': 'fecha'})
print(df_acotado)
print(df_acotado.columns)

In [ ]:
#Agrupar por año y seleccionar solo las filas del 1 de enero de cada año
df_1_enero = df_acotado[df_acotado['fecha'].dt.month == 1].groupby(df_acotado['fecha'].dt.year)
# Seleccionar las primeras 24 filas de cada grupo
df_24h_enero = df_1_enero.head(25)

# reset index
df_24h_enero.reset_index(drop=True,inplace=True)
# Reindexar con nuevo rango de índices:
df_24h_enero =df_24h_enero.reindex(range(1, len(df_24h_enero)+1  ))
 #Eliminar las últimas 2 filas con .iloc[:N]
df_24h_enero = df_24h_enero.iloc[:-2]
display(df_24h_enero)

In [ ]:
print(df_24h_enero)
# Crear un objeto para graficar los datos
fig, ax = plt.subplots()

# Graficar los datos
arr=df_24h_enero['demanda'].values

ax.plot(arr )

# Agregar etiquetas a los ejes
# Seleccionar las fechas que se mostrarán en la etiqueta del eje x
# Agregar las etiquetas del eje x
ax.set_xlabel('fecha')
ax.set_ylabel('Demanda')

# Agregar un título al gráfico
ax.set_title('Título del gráfico')

# Mostrar el gráfico
plt.show()

In [ ]:
df_24h_enero.describe(include='all')

In [ ]:
print(f'Time period start : {df_24h_enero.fecha.min()}\nTime period end : {df_24h_enero.fecha.max()}')

In [ ]:
df_24h_enero.columns,df_24h_enero.shape
df_24h_enero

In [ ]:
# Converting year column to datetime format
df_24h_enero['fecha'] = pd.to_datetime(df_24h_enero['fecha'], format = '%Y-%m-%d-%h')
df_24h_enero

In [ ]:
# Setting year as index for easier manipulations
y = df_24h_enero.set_index('fecha')
y,y.index,y

In [ ]:
print(y)
# Crear un objeto para graficar los datos
fig, ax = plt.subplots()

# Graficar los datos
arr=y['demanda'].values

ax.plot(arr )

# Agregar etiquetas a los ejes
# Seleccionar las fechas que se mostrarán en la etiqueta del eje x
# Agregar las etiquetas del eje x
ax.set_xlabel('fecha')
ax.set_ylabel('Demanda')

# Agregar un título al gráfico
ax.set_title('Título del gráfico')

# Mostrar el gráfico
plt.show()

In [ ]:
# Graficar los datos directamente sin crear fig, ax
plt.figure(figsize=(15, 6))  # Tamaño de la figura

# Graficar los datos utilizando plot
plt.plot(y.index, y['demanda'], label='Demanda', color='blue')

# Etiquetas de los ejes
plt.xlabel('Fecha')
plt.ylabel('Demanda')

# Título del gráfico
plt.title('Título del gráfico')

# Agregar leyenda
plt.legend()

# Mostrar el gráfico
plt.show()


In [ ]:
# Null values check
y.isnull().sum()
y

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Crear una figura y ejes explícitamente
fig, ax = plt.subplots(figsize=(15, 6))

# Density Plot usando Seaborn
sns.histplot(y, kde=True, ax=ax)

# Etiquetas de los ejes
ax.set_xlabel('Fecha')
ax.set_ylabel('Demanda')

# Mostrar el gráfico
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Crear una figura y ejes explícitamente
fig, ax = plt.subplots(figsize=(15, 6))

# Boxplot usando Seaborn
sns.boxplot(x=y.index, y=y['demanda'], ax=ax)

# Etiquetas de los ejes
ax.set_xlabel('Fecha')
ax.set_ylabel('Demanda')

# Mostrar el gráfico
plt.show()


In [ ]:
help(sm.tsa.seasonal_decompose)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 18,8
decomposition = sm.tsa.seasonal_decompose(y, model='multiplicative',period=24)
plt.figure(figsize = (18,8))
decomposition.plot()
plt.show()

In [ ]:
# ACF & PACF Plots
plt.figure()
plt.subplot(211)
plot_acf(y['demanda'], ax=plt.gca(), lags = 30)
plt.subplot(212)
plot_pacf(y['demanda'], ax=plt.gca(), lags = 30)
plt.show()

In [ ]:
# Rolling Mean & Rolling Standard Deviation
rolmean = y.rolling(window = 24).mean() # Calcula con periodo windows la media movil 
rolstd = y.rolling(window = 24).std() # caclula la desviacino estandar movil segun la  windows 

plt.figure(figsize = (15,6))
orig = plt.plot(y, color = 'blue', label ='Original')
mean  = plt.plot(rolmean, color = 'red', label = 'Rolling Mean')
std = plt.plot(rolstd, color = 'black', label = 'Rolling Std. Dev.')
plt.legend(loc = 'best')
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))

# Crear una figura y ejes explícitamente
fig, ax = plt.subplots(figsize=(15, 6))

# Graficar la serie temporal original
orig = ax.plot(y, color='blue', label='Original')

# Graficar la media móvil
mean = ax.plot(rolmean, color='red', label='Rolling Mean')

# Graficar la desviación estándar móvil
std = ax.plot(rolstd, color='black', label='Rolling Std. Dev.')

# Agregar leyenda
ax.legend(loc='best')

# Mostrar el gráfico
plt.show()


In [ ]:
# Creating general function to test stationarity of a time series
   
def test_stationarity(timeseries):
    B = timeseries.iloc[:, 0]
    B = B.reset_index(drop=True)
    # Rolling Mean & Rolling Standard Deviation
    rolmean = timeseries.rolling(window = 24).mean()
    rolstd = timeseries.rolling(window = 24).std()

    plt.figure(figsize = (15,6))
    orig = plt.plot(timeseries, color = 'blue', label ='Original')
    mean  = plt.plot(rolmean, color = 'red', label = 'Rolling Mean')
    std = plt.plot(rolstd, color = 'black', label = 'Rolling Std. Dev.')
    plt.legend(loc = 'best')
    plt.show()
    
    # Augmented Dicky-Fuller Test
    print('-------------Results of Dicky Fuller Test -------------')
    dftest = adfuller(timeseries, autolag = 'AIC')
    dfoutput = pd.Series(data = dftest[0:4], index = ['Test Statistic : adf', 'p-value : MacKinnon\'s approximate p-value',
                                                     'No. of Lags used', 'No. of observations used'])
    for key,value in dftest[4].items():
        dfoutput[f'Critical Value ({key})'] = value
    dfoutput['Maximized AIC:'] = dftest[5]
    print(dfoutput)
    if dftest[1]>0.05 :
            print("the null hypothesis is fulfilled for no stationary series ")
    else:
        print("the  hypothesis is fulfilled for  stationary series ")
    
     #! Agree the Levene test
    # Divide the data into three equal parts
    
    if len(B)//2==0 :
        part_size = len(B) // 2
        seg1 = B[:part_size]
        seg2 = B[2*part_size:]
            # # Apply the levene test to each pair of segments
        resultado_levene_seg1_seg2 = levene(seg1, seg2)
    # Apply the levene test to each pair of segments
         # # Print the results
        print("Valor p de la prueba de levene para seg1 y seg2:", resultado_levene_seg1_seg2.pvalue)
        if resultado_levene_seg1_seg2.pvalue < 0.05:
            print('La serie no es estacionaria en VARIANZA')
        else:
            print('La serie es estacionaria en VARIANZA')
    else:
    # Adjust the parts size if needed to make them approximately equal
        part_size = len(B) // 3
        # Divide the series into three parts
        seg1 = B[:part_size]
        seg2 = B[part_size:2*part_size]
        seg3 = B[2*part_size:]
        print(seg3)
        # Apply the levene test to each pair of segments
        resultado_levene_seg1_seg2 = levene(seg1, seg2)
        resultado_levene_seg2_seg3 = levene(seg2, seg3)
        resultado_levene_seg3_seg1 = levene(seg3, seg1)

        # Print the results
        print("Valor p de la prueba de levene para seg1 y seg2:", resultado_levene_seg1_seg2.pvalue)
        print("Valor p de la prueba de levene para seg2 y seg3:", resultado_levene_seg2_seg3.pvalue)
        print("Valor p de la prueba de levene para seg3 y seg1:", resultado_levene_seg3_seg1.pvalue)

        if resultado_levene_seg1_seg2.pvalue < 0.05 and resultado_levene_seg2_seg3.pvalue < 0.05 and resultado_levene_seg3_seg1.pvalue < 0.05:
            print('La serie no es estacionaria en VARIANZA')
        else:
            print('La serie es estacionaria en VARIANZA')


In [ ]:
# Stationarity test of log transformed TS
test_stationarity(y)

In [ ]:
# Splitting TS data into train and test set for model training and testing


train_ts = y.iloc[: 120,]
test_ts = y.iloc[120: ,]
test_ts.isnull().sum()
print(len(y))
len(test_ts)

In [ ]:
# Auto arima: seleccion basada en AIC
# ==============================================================================
modelo = auto_arima(
            y                 = test_ts,
            start_p           = 0,
            start_q           = 0,
            max_p             = 3,
            max_q             = 3,
            seasonal          = True,
            test              = 'adf',
            m                 = 24, # periodicidad de la estacionalidad
            d                 = None, # El algoritmo determina 'd'
            D                 = None, # El algoritmo determina 'D'
            trace             = True,
            error_action      = 'ignore',
            suppress_warnings = True,
            stepwise          = True,
            random_state      =42,
            n_fits            =15000
)
print(modelo.summary())


In [ ]:

model = ARIMA(train_ts, order = (1,0,0), seasonal_order=(2,0,0,24))
results = model.fit()
print(results.summary())

In [ ]:
# Obtén las predicciones para el conjunto de prueba
# predictions = modelo.predict(n_periods=24)
predictions_test_=modelo.predict(len(test_ts))
# Crea un DataFrame con las fechas y las predicciones
predictions_df = pd.DataFrame(index=test_ts.index)
predictions_df['Predictions'] = predictions_test_.values  # Asegúrate de extraer los valores

# Asegúrate de que el índice del DataFrame sea de tipo datetime
predictions_df.index = pd.to_datetime(predictions_df.index)

# Imprime el DataFrame con las predicciones
print(predictions_df)

# Describe el DataFrame con las predicciones
predictions_df.describe()


In [ ]:
plt.figure(figsize = (15,6))
plt.plot(test_ts, color = 'green', label = 'Log Transformed Original data')
plt.plot(predictions_df, color = 'blue', label = 'Predicted values for train dataset')
plt.plot(predictions_df, color = 'orange', label = 'Predicted values for test dataset')
plt.xlabel('Month')
plt.ylabel('Log of no. of passengers')
plt.title('ARIMA(3,2,2)(0,1,0)[12] qualitative performance')
plt.legend(loc = 'best')
plt.show()